In [1]:
import sys
import asyncio
import time
import os
import numpy as np
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta

In [2]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
#atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(latiss.start_task)

atcamera: Adding all resources.


Disabling authlist-based command authorization


atspectrograph: Adding all resources.


Disabling authlist-based command authorization


atheaderservice: Adding all resources.


...

Read 1 history items for RemoteEvent(ATSpectrograph, 0, softwareVersions)


Read 13 history items for RemoteEvent(ATSpectrograph, 0, summaryState)


Read historical data in 0.17 sec


Read 100 history items for RemoteEvent(ATCamera, 0, heartbeat)


[[None, None, None, None]]

In [ ]:
# enable components
#await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable()

In [ ]:
# ATCamera didn't come up
tmp=await latiss.rem.atcamera.cmd_start.set_start(timeout=10)
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

In [ ]:
# enable components
#await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable()

In [ ]:
# ATArchiver also didn't come up
tmp=await latiss.rem.atarchiver.cmd_start.set_start()
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.ENABLED)

In [ ]:
# All up now

In [ ]:
# Take a bias to see if everything is working.
await latiss.take_bias(1)

In [ ]:
# Take 10 biases 
# Added wait to stop killing the recent images
for i in range(10):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
# Take a flat to see if everything is working.
await latiss.take_flats(2.0, 1)

In [ ]:
# Now try the OCS version of the shifting

In [8]:
# Needed this because the set below failed in the middle
await latiss.rem.atcamera.cmd_disableCalibration.set_start()

logMessage DDS read queue is filling: 11 of 100 elements


logMessage DDS read queue is filling: 18 of 100 elements


In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=100.0)
await latiss.rem.atcamera.cmd_startImage.start()
for i in range(4):
    await asyncio.sleep(2.0)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=500)
    await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(2.0)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=100.0)
await latiss.rem.atcamera.cmd_startImage.start()
for i in range(8):
    await asyncio.sleep(2.0)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=100)
    await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(2.0)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


In [5]:
# This is the one to use
# HD48115
# Seq #a 213 and 214
exptime = 2.0
expcount = 20
rowshift = 100
buffer = 30.0 # Time buffer for timeout
timeout = exptime * expcount + buffer

await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=timeout)
await latiss.rem.atcamera.cmd_startImage.start()
for i in range(expcount - 1):
    print(f"Exposing {exptime} seconds.")
    await asyncio.sleep(exptime)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=rowshift)
    print(f"Shifting {rowshift} rows.")
    await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(exptime)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


groupId: 2022-03-17T23:47:02.913,imageType: ENGTEST
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.
Exposing 2.0 seconds.
Shifting 100 rows.


In [6]:
# Tried running donuts on HD48115
# Why did this fail?

exptime = 10.0
expcount = 10
rowshift = 200
buffer = 30.0 # Time buffer for timeout
timeout = exptime * expcount + buffer

await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=timeout)
await latiss.rem.atcamera.cmd_startImage.start()
for i in range(expcount - 1):
    print(f"Exposing {exptime} seconds.")
    await asyncio.sleep(exptime)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=rowshift)
    print(f"Shifting {rowshift} rows.")
    await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(exptime)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


groupId: 2022-03-17T23:50:55.515,imageType: ENGTEST


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1839112488, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid argument')

In [8]:
# Needed this because the set above failed
await latiss.rem.atcamera.cmd_disableCalibration.set_start()

In [7]:
# Take a flat to see if everything is working.
await latiss.take_flats(2.0, 1)

Generating group_id


imagetype: FLAT, skip TCS synchronization.


FLAT 0001 - 0001


array([2022031700216])

In [ ]:
# Doing it step-by-step.  This is image 29
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [ ]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"

In [ ]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=100.0)

In [ ]:
await latiss.rem.atcamera.cmd_startImage.start()

In [ ]:
await asyncio.sleep(10.0)

In [ ]:
latiss.rem.atcamera.cmd_discardRows.set(nRows=500)

In [ ]:
await latiss.rem.atcamera.cmd_discardRows.start()

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.set_start()

In [ ]:
# Take a flat to see if everything is working.
await latiss.take_flats(10.0, 1)

In [ ]:
await latiss.standby()